In [1]:
%%html
<link rel="stylesheet" href="//maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anonymous">


<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Intro to Spark

_Authors: David Yerrington (SF)_

---

![](https://snag.gy/ieVW98.jpg)

### Learning Objectives
- Identify major data types used in Spark.
- Identify basic operations for data munging in Spark.
- Understand the Spark computation model.
- Use Spark UI to view running "jobs."
- Build simple queries and transformations for Spark via Python.

### Student Pre-Work
*Before this lesson, you should already be able to:*
- Install and configure the latest version of Apache Spark.
- Configure Jupyter Notebook with SparkContext.


### Lesson Guide
- [What is Spark?](#intro)
    - [Spark Is a Distributed Framework for Parallelized Applications](#dist)
    - [Spark Is an API](#api)
    - [Spark Is Machine Learning](#ml)
    - [Spark Is a Framework for Building High-Volume Stream Processors](#stream)
    - [Spark Is a SQL Interface](#sql)
    - [Spark Is Parallel Graph Processing](#graph)
- [Programming With Spark](#prog)
- [Spark UI](#sparkui)
- [Using Spark via PySpark](#using)
    - [Spark Installation Guide](#guide)
    - [The SparkContext](#spark-context)
- [RDDs vs. DataFrames](#rdd)
- [Transformations and Actions](#ta)
    - [Common Spark Transformations](#common-transformations)
    - [Common Spark Actions](#common-actions)
- [Spark Data Types](#dtypes)
    - [Resilient Distributed Data Set](#rdds)
    - [DataFrames](#df)
- [Common DataFrame Operations and Characteristics](#common-df)
- [Some Basic Statistics in Spark](#stats)
- [Limiting Results](#limiting)
- [More DataFrame and Series Operations](#more-ops)
- [Activity: Check Out Another Data Set Using Spark](#activity)
- [Practical Tips](#practical-tips)
- [Additional Resources](#additional-resources)

<a id='intro'></a>
## What Is Spark?
---

**How is it different and similar to:**

- Hadoop?
- MapReduce?
- Random forests?


### What Is This "Spark" You Speak of?

Apache Spark is an open-source cluster computing framework.

Spark provides an interface for programming entire computer clusters with implicit data parallelism and fault tolerance.

Spark has gained traction over the past few years because of its superior performance with respect to Hadoop-MapReduce.

Spark relaxes the constraints of MapReduce by doing the following:

- It generalizes computation from MapReduce by only graphing to arbitrary directed acyclic graphs (DAGs).
- It removes a lot of the boilerplate code present in Hadoop.
- It can "tweak" things that are not accessible in Hadoop (e.g., the sort algorithm).
- It can load data in a cluster memory, greatly speeding up I/O. 

![](https://snag.gy/4oxeiA.jpg)

- **Spark Core**: Contains the basic functionality of Spark; in particular, the APIs that define resilient distributed data sets (RDDs) and the operations and actions that can be undertaken upon them. The rest of Spark's libraries are built on top of the RDD and Spark Core.

- **Spark SQL**: Provides APIs for interacting with Spark via the Apache Hive variant of SQL called Hive query language (HiveQL). Every database table is represented as an RDD, and Spark SQL queries are transformed into Spark operations. For those that are familiar with Hive and HiveQL, Spark can act as a drop-in replacement.

- **Spark Streaming**: Enables the processing and manipulation of live streams of data in real time. Many streaming data libraries (such as Apache Storm) exist for handling real-time data. Spark Streaming enables programs to leverage this data similarly to how you would interact with a normal RDD as data are flowing in.

- **MLlib**: A library of common machine learning algorithms implemented as Spark operations on RDDs. This library contains scalable learning algorithms, such as classifications and regressions, that require iterative operations across large data sets. The Mahout library — formerly the big data machine learning library of choice — will move to Spark for its implementations in the future.

- **GraphX**: A collection of algorithms and tools for manipulating graphs and performing parallel graph operations and computations. GraphX extends the RDD API to include operations for manipulating graphs, creating sub-graphs, or accessing all vertices in a path.

### Why Use Spark?

Imagine you have to run a linear regression over several terabytes of data. A few obvious problems arise:

- It's hard to find a place to store all of that data.
- It would take a painfully long time to compute the regression.

Spark helps solve these problems by allowing us to distribute the data over several computers and use main memory to speed up the computation. Main memory runs about 1,000 times faster than hard disk, but real performance is more complex than just those two components.

Spark is also often used for streaming data. Perhaps a company needs instant information about a large number of incoming tweets.

### Spark Is Many Things  

- It's a set of tools for developing applications.  
- It's the parallel processing of applications in a distributed environment.

![](https://snag.gy/c9b1Kx.jpg)


<a id='dist'></a>
### Spark Is a Distributed Computer Framework for Parallelized Applications Like _Hadoop_

_Spark can interact with Hadoop's HDFS to access large amounts of data using high-volume, distributed I/O._

![](https://snag.gy/s8gSlG.jpg)

<a id='api'></a>
### Spark Is an API for Handling Large Data Transformations Like _MapReduce_

_Spark is a data transformation and selection tool similar to **Pandas**. You can develop transformations through an API that allows you to chain operations together in a modular fashion._

![](https://snag.gy/9G4gJO.jpg)

> However, Spark delivers on the idea of data manipulation through the framework of **transformations** and **actions**. These transformations and actions are performed parallely using many different worker nodes (which may be distributed on multiple machines).

> MapReduce is broken down into two main functions: map and reduce. Spark, on the other hand, runs through a set of operations determined through a **directed acyclic graph** (DAG).

<a id='ml'></a>
### Spark Is Machine Learning Like _Scikit-Learn_

![](https://snag.gy/RnuX6h.jpg)

_Spark provides an interface to MLib via Scala, Java, Python, and R. The most common methods are provided, including regression, support vector machines, and random forests. However, not all evaluation metrics are available in Python yet. Spark is written in Scala, so features are prioritized to Scala first throughout the Spark ecosystem._

> <i class="fa fa-question-circe"></i> Does anyone remember specific limitations? (You might not if you haven't done MLlib with Spark yet.)

<a id='stream'></a>
### Spark Is a Framework for Building High-Volume Stream Processors
![](https://snag.gy/RCikuU.jpg)

With Spark Streaming, it's possible to build a process that can respond to data in real time using any of Spark's features — including MLib, GraphX, or any transformations you could do within the SparkContext. The streaming capabilities of Spark Core make it possible to produce real-time applications, such as ETL, analytics dashboards, data mining, or large-scale aggregations.

In short, you can create a "streaming context" that listens to a specific port and tie it to any number of operations that can be programed with Spark.

>```python
># Save this as a file called "network_streaming.py."
>from pyspark import SparkContext
>from pyspark.streaming import StreamingContext
>
># Create a local StreamingContext with two working threads and a batch interval of one second.
>sc = SparkContext("local[2]", "NetworkWordCount")
>ssc = StreamingContext(sc, 1)
>
># Create a DStream that will connect to hostname:port (i.e., localhost:9999).
>lines = ssc.socketTextStream("localhost", 9999)
>
># Split each line into words.
>words = lines.flatMap(lambda line: line.split(" "))
>
># Count each word in each batch.
>pairs = words.map(lambda word: (word, 1))
>wordCounts = pairs.reduceByKey(lambda x, y: x + y)
>
># Print the first 10 elements of each RDD generated in this DStream to the console.
>wordCounts.pprint()
>
>ssc.start()             # Start the computation.
>ssc.awaitTermination()  # Wait for the computation to terminate.
>```

Then, you can have Spark run this stream using the UNIX utility **netcat** to route your service once you run `spark-submit`.

>```bash
>$ nc -lk 9999
>```

Running `spark-submit` launches applications onto your Spark cluster.

>```bash
>./bin/spark-submit examples/src/main/python/streaming/network_wordcount.py localhost 9999
>```

<a id='sql'></a>
### Spark Is a SQL Interface Into DataFrames Like _Hive_

Spark isn't actually **Hive**, but it uses components from it. However, you can use Spark DataFrames more easily with temporary SQL views.

>```python
># Load a data set as a Spark DataFrame.
>df = spark.read.csv("datasets/somedataset/hamburgers_eaten_per_hour.csv")
>df.createOrReplaceTempView("hamburgers")
>```



Then, viola! You can slice and dice your DataFrame as SQL:

>```python
>spark.sql("SELECT * FROM hamburgers").show()
>
># +------+---------+
># | eaten|     name|
># +------+---------+
># |null  |     Jeff|
># |  30  |   Kiefer|
># |  19  |     Hang|
># +------+---------+
>```

<a id='graph'></a>
### Spark Is Also Parallel Graph Processing

![](https://snag.gy/0KvrCw.jpg)

Graph processing provides the ability to process data in terms of relationships. Traditional relational database management systems (RDBMS) fail to calculate relationships between entities that don't have a predetermined depth between sets.  

Using graphs, you can represent irregular relationship shapes within data and apply functions recursively and/or arbitrarily on any depth of relationships.

Both RDBMS and graphs can be used to equate the other's feature, but each have their strengths and trade-offs, depending on the application.


<a id='prog'></a>
## Programming With Spark
---

There are a variety of ways to run application builds with Spark, including:

- PySpark
- Spark-submit

### PySpark
PySpark is a real-time interpreter that communicates between Spark to Python and allows you to integrate operations with the Spark Python libraries. This is a great way to prototype applications much like we would with Jupyter Notebook. It's also possible to connect PySpark to Jupyter.

### Spark-Submit
Spark-submit, on the other hand, is a way to run a set of application instructions bundled in a single file. It's possible to write these in Python, Scala, or Java. Typically, it's convenient to prototype a set of operations that you want performed on a data set with PySpark, debug them to a high level of quality, and then run them with spark-submit.

> With spark-submit, it's also possible to tune your application's parameters to a granular degree, including memory, number of total cores, and specific cluster modes to control test versus production deployments.
>
> [Read more about submitting applications](http://spark.apache.org/docs/latest/submitting-applications.html).

<a id='sparkui'></a>
## Spark UI
---

Anytime a SparkContext is created, a corresponding Spark UI is launched. Whenever you launch a Spark standalone instance — PySpark — a Spark UI will be created. Then, through the web UI, you can monitor how your applications run. Anything that the SparkContext handles, even the one line operations from PySpark, can be observed as a separate job in Spark UI.

### Spark Jobs
![](https://snag.gy/JnuSKC.jpg)

### Job Metrics
![](https://snag.gy/JW2fOb.jpg)

<a id='using'></a>
## Using Spark via PySpark
---

If you haven't done so yet, it's nice to be able to install Spark manually to get familiar with how to use it locally. The advantage of this, as opposed to using a virtual machine, is that you should be able to run Spark without the overhead of running an additional virtual operating system.

<a id='guide'></a>
### Spark Installation Guide
> This is a good time to review or install Spark using our guide:
>
>[Spark Installation Guide](./spark-installation-guide.md)

<a id='spark-context'></a>
### PySpark Is All About SparkContext

In [2]:
# sc

SparkContext, **`sc`**, represents your interface to a running Spark cluster manager. A SparkContext is defined as a preconfigured cluster with an application name connected to it. All **transformations** and **actions** performed by Spark are handled through the SparkContext.

>**Context for standalone Spark apps:**
>
>If we were to write a standalone app for spark-submit, we would need to create the context manually.

>```python
>from pyspark import SparkContext
>sc = SparkContext("local", "Simple App")
>```

>Otherwise, it's already created for us as a result of running **`pyspark`** from the shell, as well as in PySpark and with a connected Jupyter Notebook.

<img src="https://snag.gy/iCm4G1.jpg" width="600">

<a id='rdd'></a>
## RDDs vs. DataFrames

---

The two main types of data objects in Spark are the **resilient distributed dataset (RDD)** and the **DataFrame**. Both types represent data in a distributed state. RDDs store data in a more primitive state, such as a list of pairs, integers, floats, or strings. DataFrames have a rich structure definition called a **schema**, much like a Pandas DataFrame.

- You use **RDDs** to manage semi-structured data.
- You use **DataFrames** to operate on a typed series.

Both RDDs and DataFrames can contain multiple types of objects. **DataFrames** are much more constrained because data are illustrated by a two-dimensional tabular structure in which columns represent variables and rows represent observations. **RDDs** are much more flexible if your data require less structure than a DataFrame but you still need to be able to use _transformation_ methods (i.e., `map()`) and _action_ methods (i.e., `reduce()`).

>_Distributed data in Spark:_
>![](https://snag.gy/vxVhri.jpg)

<a id='ta'></a>

## _Transformations_ and _Actions_

---

<a id='common-transformations'></a>
### <i class="fa fa-cogs" aria-hidden="true"></i> Common Spark Transformations


Generally, **transformations** in Spark modify data as new copies of whichever data set is used as input. The default documents refer to _map_ as a **transformation** and _reduce_ as an **action**. **Transformations are lazy**, so no computations are performed until an **action** is requested.

<style>
.no-border:table, th, td {
    border: none;
    border-left: none;
    margin: 10px;
}
</style>

<table class="no-border" style="border: none;">
<tbody class="no-border" style="border: none;"><tr class="no-border" style=" background: #000000; color: #FFFFFF;"><th>Transformation</th><th style="border: none; text-align: center;">Meaning</th></tr>
<tr style="border-left: none; border-right: none;">
  <td style="border: none;"> <b>map</b>(<i>func</i>) </td>
  <td style="border-right: none;"> Returns a new distributed data set formed by passing each element of the source through a function. <i>func</i>. </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>filter</b>(<i>func</i>) </td>
  <td style="border-right: none;"> Returns a new data set formed by selecting those elements of the source on which <i>func</i> returns true. </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>sample</b>(<i>withReplacement</i>, <i>fraction</i>, <i>seed</i>) </td>
  <td style="border-right: none;"> Samples a <i>fraction</i> of the data with or without replacement using a given random number generator seed. </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>distinct</b>([<i>numTasks</i>])) </td>
  <td style="border-right: none;"> Returns a new data set that contains the distinct elements of the source data set.</td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>groupByKey</b>([<i>numTasks</i>]) <a name="GroupByLink"></a> </td>
  <td style="border-right: none;"> When called on a data set of key-value, (K, V), pairs, returns a data set of (K, Iterable&lt;V&gt;) pairs. <br>
    <b>Note:</b> If you're grouping in order to perform an aggregation (such as a sum or
      average) over each key, using <code>reduceByKey</code> or <code>aggregateByKey</code> will yield a much better
      performance.
    <br>
    <b>Note:</b> By default, the level of parallelism in the output depends on the number of partitions of the parent RDD.
      You can pass an optional <code>numTasks</code> argument to set a different number of tasks.
  </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>reduceByKey</b>(<i>func</i>, [<i>numTasks</i>]) <a name="ReduceByLink"></a> </td>
  <td style="border-right: none;"> When called on a data set of (K, V) pairs, returns a data set of (K, V) pairs where the values for each key are aggregated using the given reduce function <i>func</i>, which must be of type (V, V) =&gt; V. The number of reduce tasks is configurable through an optional second argument, as with <code>groupByKey</code>. </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>aggregateByKey</b>(<i>zeroValue</i>)(<i>seqOp</i>, <i>combOp</i>, [<i>numTasks</i>]) <a name="AggregateByLink"></a> </td>
  <td style="border-right: none;"> When called on a data set of (K, V) pairs, returns a data set of (K, U) pairs where the values for each key are aggregated using the given combine functions and a neutral zero value. Allows an aggregated value type that is different than the input value type while avoiding unnecessary allocations. Like in <code>groupByKey</code>, the number of reduce tasks is configurable through an optional second argument. </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>sortByKey</b>([<i>ascending</i>], [<i>numTasks</i>]) <a name="SortByLink"></a> </td>
  <td style="border-right: none;"> When called on a data set of (K, V) pairs where K implements ordered, returns a data set of (K, V) pairs sorted by keys in ascending or descending order, as specified in the Boolean <code>ascending</code> argument.</td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>join</b>(<i>otherDataset</i>, [<i>numTasks</i>]) <a name="JoinLink"></a> </td>
  <td style="border-right: none;"> When called on data sets of type (K, V) and (K, W), returns a data set of (K, (V, W)) pairs with all pairs of elements for each key.
    Outer joins are supported through <code>leftOuterJoin</code>, <code>rightOuterJoin</code>, and <code>fullOuterJoin</code>.
  </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>pipe</b>(<i>command</i>, <i>[envVars]</i>) </td>
  <td style="border-right: none;"> Pipes each partition of the RDD through a shell command (e.g., a Perl or Bash script). RDD elements are written to the process' `stdin`, and lines that output to its `stdout` are returned as an RDD of strings. </td>
</tr>
</tbody></table>

> <i class="fa fa-info-circle" aria-hidden="true"></i> For more info on **transformations**, check out the [<i class="fa fa-external-link" aria-hidden="true"></i> Spark programming guide on #transformations](http://spark.apache.org/docs/latest/programming-guide.html#transformations).

> A great visual guide for common Spark transformations is available from [<i class="fa fa-external-link" aria-hidden="true"></i> Jeff Thompson at Databricks](http://training.databricks.com/visualapi.pdf).

<a id='common-actions'></a>
### <i class="fa fa-wrench" aria-hidden="true"></i> Common Spark Actions

Spark **actions** aren’t performed until they are called — sounds like common sense, right? However, be careful, because in _Pandas_ we're accustomed to getting results immediately after most transformations are performed. In Spark, even if we have a few operations programmed out, nothing will actually be transformed until we call an **action**.

```python
# 1) Read text file.
#########
# This line reads a text file from the file system. Each line becomes an element in an RDD.
# No operations have been performed yet. We don't know if the text file actually exists or if it read any data.
#############################################################################################################

text_lines = sc.textFile("somefile.txt")

# 2) Map lengths of each line.
#########
# Once data are available, we simply count the length of lines (length of string) and create a new RDD,
# which only has the length of each line as a new object.
#############################################################################################################

text_line_lengths = text_lines.map(lambda s: len(s))

# 3) Reduce to find the total sum of lengths.
#########
# Once reduce is called, all prior operations are run, meaning we actually run sc.textFile() 
# and text_lines.Map() before finally running text_line_lengths.reduce().
#############################################################################################################

total_length = text_line_lengths.reduce(lambda a, b: a + b)
```

<table class="table" style="border-left: none; border-right: none;">
<tbody style="border-left: none;"><tr style="background: black; color: white;"><th>Action</th><th style="text-align: center;">Meaning</th></tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>reduce</b>(<i>func</i>) </td>
  <td style="border-right: none;"> Aggregates the elements of the data set using a function, <i>func</i> (which takes two arguments and returns one). The function should be commutative and associative so that it can be computed correctly in parallel. </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>collect</b>() </td>
  <td style="border-right: none;"> Returns all of the elements of the data set as an array at the driver program. This is usually useful after a filter or other operation that returns a sufficiently small subset of the data. </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>count</b>() </td>
  <td style="border-right: none;"> Returns the number of elements in the data set. </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>first</b>() </td>
  <td style="border-right: none;"> Returns the first element of the data set (similar to `take(1)`). </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>take</b>(<i>n</i>) </td>
  <td style="border-right: none;"> Returns an array with the first <i>n</i> elements of the data set. </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>takeSample</b>(<i>withReplacement</i>, <i>num</i>, [<i>seed</i>]) </td>
  <td style="border-right: none;"> Returns an array with a random sample of <i>num</i> elements from the data set — with or without replacement — optionally pre-specifying a random number generator seed.</td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>takeOrdered</b>(<i>n</i>, <i>[ordering]</i>) </td>
  <td style="border-right: none;"> Returns the first <i>n</i> elements of the RDD using either their natural order or a custom comparator. </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>saveAsTextFile</b>(<i>path</i>) </td>
  <td style="border-right: none; border-right: none;"> Writes the elements of the data set as a text file (or set of text files) in a given directory in the local file system, HDFS, or any other Hadoop-supported file system. Spark will call `toString()` on each element to convert it to a line of text in the file. </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>countByKey</b>() <a name="CountByLink"></a> </td>
  <td style="border-right: none;"> Only available on RDDs of type (K, V). Returns a HashMap of (K, Int) pairs with the count of each key. </td>
</tr>
<tr style="border-left: none; border-right: none;">
  <td style="border-left: none;"> <b>foreach</b>(<i>func</i>) </td>
  <td style="border-right: none;"> Runs a function <i>func</i> on each element of the data set. This is usually done for side effects such as updating an <a href="#accumulators">accumulator</a> or interacting with external storage systems.
  <br><b>Note</b>: Modifying variables other than accumulators outside of <code>foreach()</code> may result in undefined behavior. See <a href="#understanding-closures-a-nameclosureslinka">understanding closures </a> for more details.</td>
</tr>
</tbody></table>

> <i class="fa fa-info-circle" aria-hidden="true"></i> For more information on **actions**, check out the [<i class="fa fa-external-link" aria-hidden="true"></i> Spark programming guide on #actions](http://spark.apache.org/docs/latest/programming-guide.html#actions).

# <i class="fa fa-question-circle" aria-hidden="true"></i>   How Might You Approach Programming With Spark vs. Pandas? (~1 Minute)

_Given that the operations are lazy?_

<a id='dtypes'></a>
## Spark Data Types
---

### RDDs

It's best to think of RDDs as primitive, distributed objects. RDDs can contain any type of Python, Java, or Scala objects, including user-defined classes. The RDD type is the oldest type and has been around since the first version of Spark. 

> A handy trick: Because RDDs can be Python class types, it's possible to do things like run scikit-learn's `GridSearch` over an existing scikit-learn model and a parameter search over a particular amount of permutations in a clustered Spark set up.

### DataFrames

As part of the "Tungsten Initiative," which sought to improve the performance of Spark, DataFrames entered the Spark code base in version 1.3. The big difference between RDDs and DataFrames is that DataFrames introduce the idea of a "schema," much like Pandas.  

<a id='rdds'></a>
<span style="font-size: 25pt; font-weight: light;"><strong>R</strong>eslient <strong>D</strong>istributed <strong>D</strong>ata set</span>
<hr>


Everything in Spark revolves around the **RDD**.

> "...a fault-tolerant collection of elements that can be operated on in parallel..."
> _— Spark documentation_

Most data sets that can be loaded externally (CSV, HDFS/Hadoop, text files, RDBMS/SQL, etc.) become an RDD and are operated in parallel within a distributed system (Spark). Otherwise, you can `sc.parallelize(my_data)` a data set through a SparkContext (sometimes called a driver program).

In [3]:
# TBD: More examples with RDDs. We’ll explore DataFrames further for now, as they are more relatable.
# RDDs are great for custom jobs and operations that require a more open format.

> Check out this grid search with Spark package on [<i class="fa fa-external-link" aria-hidden="true"></i> GitHub: Spark-Scitkit-Learn](https://github.com/databricks/spark-sklearn).

<a id='df'></a>
### DataFrames

Spark DataFrames serialize their data at a lower-level native to Java/Scala. So, when a DataFrame is passed between nodes, it's much more performant, requiring fewer processes to handle computations. Data can be processed faster when optimized to a common format (the schema) that Spark doesn't have to convert to in order to perform tasks.

Outside of the performance optimizations introduced with a schema-based data structure, the **DataFrame API** provides a convenient set of selectors for transforming data, much like Pandas. Lastly, it's possible to create temporary views in which **DataFrames** can be queried with SQL (**SparkSQL**).

In [4]:
# df = spark.read.csv(
#     path        =   "../../../datasets/sentiment_words/sentiment_words_simple.csv",
#     header      =   True, 
#     mode        =   "DROPMALFORMED",   # Poorly formed rows in CSV are dropped rather than causing an error for the entire operation.
#     inferSchema =   True               # It's not always perfect, but it works well in most cases as of version 2.1+.
# )

In [5]:
# df.printSchema()

<a id='common-df'></a>
## Common DataFrame Operations and Characteristics
---

Let's take a look at some familiar and new functions and properties.

### Inspect the Variable/Column Space of a DataFrame

In [6]:
# df.columns

### Data Types
Inspect the schema programatically.

In [7]:
# df.dtypes

### Explain DataFrame
Show details about DataFrame type, schema, and origin of data.

In [9]:
# df.explain()

### Describe
Describe will look similar to the synonymous Pandas **describe** function.

In [13]:
# df.describe().show()

In [14]:
## Check the Pandas version here:
# df.toPandas().describe()

### printSchema
The schema is an important characteristic of a Spark DataFrame. It tells us what's possible in terms of transformation. It's also the reason DataFrames are so fast, as they're tied to a set number of classes that are serialized and optimized in Java/Scala behind the scenes.

><i class="fa fa-exclamation-triangle" aria-hidden="true"></i> The schema that we've been so excited to see is finally here to explore. 

In [15]:
# df.printSchema()

### Count
Count with caveat: This will return the count of all rows, including _non-NaN_ values. Pandas will omit these.

In [16]:
# df.count()

<a id='stats'></a>
## Some Basic Statistics in Spark
---

### Covariance
<span style="font-size: 20pt;">
$\operatorname{cov}(X,Y) = \operatorname{E}{\big[(X - \operatorname{E}[X])(Y - \operatorname{E}[Y])\big]}$
</span>
> "Covariance is a measure of the joint variability of two random variables. If the greater values of one variable mainly correspond with the greater values of the other variable, and the same holds for the lesser values — i.e., the variables tend to show similar behavior — the covariance is positive."

In [17]:
# df.cov("pos_score", "neg_score")

### Pearson Score
<span style="font-size: 20pt;">
$\rho_{X,Y}= \frac{\operatorname{cov}(X,Y)}{\sigma_X \sigma_Y} $
</span>
> The normalized version of the covariance — the correlation coefficient — however, shows by its magnitude the strength of the linear relation.

In [18]:
# df.corr("pos_score", "neg_score")

<a id='limiting'></a>
## Limiting Results

---

### Head

In [19]:
# A:

### Show Limited Results

With Pandas, we're used to the `df.head()` being a first step in exploring a data set. With Spark, this isn't exactly the case. You need to use the `df.show()` operation to explore data as a first step. While Pandas formats its DataFrame output for display in HTML tables with sensible defaults for output, when you're using Spark, you have to be a bit more specific about what you're looking at with `show()`.


> The `truncate` parameter is helpful for truncating attributes for display.

In [22]:
# A:

### <i class="fa fa-question-circle" aria-hidden="true"></i> Why Do You Need to Be Careful When Displaying Data in Spark?

Hopefully you can see why Pandas is so useful for EDA.

<a id='more-ops'></a>
## More DataFrame and Series Operations
---

### Convert From a List of Row Objects to a List of Dictionaries

In [23]:
# A:

### Selecting DataFrame Series

Selecting variables with Spark's **DataFrames API** works similarly to Pandas DataFrames. When selecting variables in Pandas, it uses a `list` object passed to a DataFrame object via `[]` brackets, like so:

>```df[['col1', 'col2']]```

The equivalent in Spark is using the `.select()` method, which takes flat parameters.

> <i class="fa fa-info-circle" aria-hidden="true"></i> DataFrames in Pandas are implemented with an overloaded brackets operator, so whenever we reference any Pandas DataFrames object with brackets, it gets passed down to a function that handles the column references as the input to a function.

#### Select All Features/Variables/Columns

In [24]:
# A:

#### Select Specific Features/Variables/Columns

In [26]:
# A:

#### Series Operations

With Pandas, you can easily create a new series that's the sum of every row in **"col1"** with **"col2”**, like so:

> `df['col1'] + df['col2']`

In Spark, we have to do this through the `select()` function.

In [27]:
# A:


#### As an "Alias"
Selections can become hard to read, as they are keyed by conditions. We can use an "alias" to abstract any selections.

In [29]:
# A:

#### Creating New Features/Variables/Columns

In [30]:
# A:

## <i class="fa fa-question-circle"></i> Have We Changed the Original DataFrame?

#### Filtering Data
In Pandas, we use masks through DataFrame object brackets in order to filter data.
>`df[df['feature'] > 0]`

In Spark, we use the `filter()` method to select different aspects of our data.

In [31]:
# A:

#### Multiple Conditions
This is actually the same in Spark as it is in Pandas. 

In [32]:
# A:

#### Filter as an Expression
Pandas has a similar function called "where." However, with Spark `filter()`, we can filter using shorthand expressions when referencing column sequences.

In [34]:
# A:

### Sorting

In [36]:
# A:

## As SQL?


Working with DataFrames as SQL is as easy as creating a **temporary view**.

In [40]:
# A:

### Temporary Views Select DataFrames
The same transformations we just learned can be applied to DataFrames.

In [43]:
# A:

<a id='activity'></a>
## Activity: Check Out Another Data Set Using Spark DataFrames

#### 1) Load up the "Pokemon" basic Pokedex data set.
First, try this without inferring the schema and without the header.

In [44]:
# A:

#### 2) Check out the data set with the` inferSchema` parameter but without `header`.
How does it work with/without the schema parameter?

In [45]:
# A:

#### 3) Create a temporary view with the Pokedex DataFrame called "pokemon."
Then 
```sql SELECT * FROM pokemon LIMIT 10```

In [46]:
# A:

#### 4.A) Which is the strongest Pokemon by `Type`?
Research Spark's "grouping" functions using the Spark DataFrame operations.

In [47]:
# A:

#### 4.B) Which is the strongest Pokemon by `Type`?
Using the Spark SQL temporary view.

In [48]:
# A:

#### 5.A) Which Pokemon has the best combined attack and defense?
Using Spark DataFrame operations.

In [49]:
# A:

#### 5.B) Which Pokemon has the best combined attack and defense?
Using the Spark SQL temporary view.

In [50]:
# A:

#### 6) Create a new feature called "Pokevalue" that is the combined attack and defense scaled by .2 of the Pokemon HP.
Use any means necessary to solve this problem.

In [51]:
# A:

<a id='practical-tips'></a>
# <i class="fa fa-thumbs-up" aria-hidden="true"></i> Practical Tips
---

### Sampling From Enormous Data Sets
Undoubtedly, you may need to examine a larger data set. A common operation is to take a sample. To approximate the characteristics of your global distribution, you should try to adjust the size that best matches the metrics of central tendency or consider performing a power analysis to determine sample sizing.

> The size of your sample generally depends on your application, be it A/B testing, EDA, machine learning, etc.

In [52]:
# A:

<a id='additional-resources'></a>
## Additional Resources

- [Qubole: Apache Spark Use Cases](https://www.qubole.com/blog/big-data/apache-spark-use-cases/)
- [Spark Examples](http://spark.apache.org/examples.html)